```{admonition} Lecture Materials
:class: tip
[Download the slide deck for this lecture](/_static/pdf/Lecture16-PartialDifferentialEquations-2.pdf)
```

# Initial value problem: Heat equation

In most cases we are studying the time evolution of a certain profile $u(t,\mathbf{x})$.
In this case PDEs describe the time evolution of the field(s) starting from some initial conditions and obeying boundary conditions.

Let us take the heat equation as an example.
The field $u$ is the temperature $T$.
In one dimension the heat equation reads

$$
\frac{\partial u}{\partial t} = D \, \frac{\partial^2 u}{\partial x^2},
$$

where $D$ is the thermal diffusivity constant.

This equation describes the time evolution of $u(t,x)$ given initial profile

$$
u(t=0,x) = u_0(x),
$$

and boundary conditions
\begin{align*}
u(t,x=0) & = u_{\rm left}(t), \\
u(t,x=L) & = u_{\rm right}(t).
\end{align*}

If $u_{\rm left}(t)$ and $u_{\rm right}(t)$ are time-independent, we know that the solution will approach a stationary profile as $t \to \infty$.

## FTCS scheme

FTCS (Finite Time Centered Space) scheme is the simplest method for solving the heat equation

First we discretize the spatial coordinate into a grid with $N + 1$ points, i.e.

$$
x_k = a k, \qquad k = 0\ldots N, \qquad a = L/N,
$$

and approximate the derivative $\partial^2 u / \partial x^2$ by the lowest order central difference

$$
\frac{\partial^2 u(t,x)}{\partial x^2} \approx \frac{u(t,x+a) - 2u(t,x) + u(t,x-a)}{a^2}.
$$

To evaluate the time evolution we will work with small time steps of size $h$.
The time derivative is approximated by the forward difference

$$
\frac{\partial u(t,x)}{\partial t} \approx \frac{u(t+h,x) - u(t,x)}{h}.
$$

This gives the following discretized equation

$$
\frac{u(t+h,x) - u(t,x)}{h} = D \frac{u(t,x+a) - 2u(t,x) + u(t,x-a)}{a^2}.
$$

The method is explicit: to evaluate $u(t+h,x)$ at the next time step we only need to know $u(t,x)$ profile at the present time step.
Denoting the discretized time variable by superscript $n$ (such that $t_n = hn$) and the spatial variable by subscript $k$ (such that $x_k = ak$) we get the following iterative procedure

$$
u^{n+1}_k = u^n_k + r \, (u^n_{k+1} - 2u^n_k + u^n_{k-1}), \qquad k = 1 \ldots N-1.
$$

Here

$$
r \equiv \frac{Dh}{a^2}
$$

is a dimensionless parameter.

### Implementation of the FTCS scheme

Let us implement the FTCS scheme in Python

In [1]:
using Plots
gr()

Plots.GRBackend()

In [ ]:
function heat_FTCS_iteration(u,r)
	N = length(u) - 1
	u_new = zeros(length(u))
	u_new[1] = u[1]
	u_new[N+1] = u[N+1]
	for i in 2:N
		u_new[i] = u[i]+r*(u[i+1]-2*u[i]+u[i-1]) 
	end
	return u_new
end

function heat_FTCS_solve(u₀,t,N,L,D)
	Δt = t[2]-t[1]
	Δx = L/N
	r = D*Δt/Δx^2
	println("r = $r")
	u_sol = zeros(length(t),(length(u₀)))
	u = copy(u₀)
	u_sol[1,:] = u
	for i in 1:length(t)-1
		u = heat_FTCS_iteration(u,r)
		u_sol[i+1,:] = u
	end
	return u_sol
end

heat_FTCS_solve (generic function with 1 method)

### Example

Let us consider the following problem: Example 9.3 from M. Newman, *Computational Physics*:

We have a 1 cm long steel container, initially at a temperature 20&deg; C.
It is placed in bath of cold water at 0&deg; C and filled on top with hot water at 50&deg; C.
Our goal is to calculate the temperature profile as function of time.
The thermal diffusivity constant for stainless steel is $D = 4.25 \cdot 10^{-6}$ m$^2$ s$^{-1}$.

We will calculate the profile at times $t = 0.01$ s, $0.1$ s, $0.4$ s, $1$ s, and $10$ s.

In [59]:
@time begin
	# Constants
	Tleft = 50
	Tright = 0
	Tmiddle = 20
	L = 0.1      # Thickness of steel in meters
	D = 4.25e-5   # Thermal diffusivity
	N = 10      # Number of divisions in grid
	Δx = L/N       # Grid spacing
	x = 0:Δx:L   # Spatial grid
	Δt = 1e-3      # Time-step (in s)
	u = zeros(N+1)
	u[1] = Tleft
	u[N+1] = Tright
	u[2:end-1] .= Tmiddle
	r = D*Δt/Δx^2
	t_end = 10
	t = 0:Δt:t_end

    u_sol_explicit = heat_FTCS_solve(u,t,N,L,D);
end;

r = 0.00042500000000000003
  0.000700 seconds (20.04 k allocations: 2.215 MiB)


In [ ]:

fps = 100
Δi=round(Int, 1/ (fps*Δt))
anim = @animate for i in 1:Δi:length(t)
    p1 = heatmap(x, [1], u_sol_explicit[i, :]', xlabel="Postion", clim=(0,50))
    p2 = plot(x,u_sol_explicit[i, :],xlabel="x", ylabel="T [C°]",legend=false)
    plot(p1, p2, layout = (1,2), size=(1000,400),title="t=$(round(t[i], digits=1))")
end
mp4(anim, fps=fps)

┌ Info: Saved animation to c:\Users\dvnoe\OneDrive - HAN\Documenten\GitHub\Computational-physics\book\10_PDE\heat.mp4
└ @ Plots C:\Users\dvnoe\.julia\packages\Plots\8ZnR3\src\animation.jl:156


Plots.AnimatedGif("c:\\Users\\dvnoe\\OneDrive - HAN\\Documenten\\GitHub\\Computational-physics\\book\\10_PDE\\heat.mp4")

The method has become unstable.
What if we increase the spatial size step to bring $r$ back below 1/2?

For $r < 1/2$ the method is stable again.

This underlines the importance of the stability condition for explicit methods.
Depending on the relation between time and space steps, we can either have a stable or unstable method.
For heat equation the stability condition is

$$
r = \frac{\Delta t D}{\Delta x^2} < 1/2
$$


## Implicit scheme

In the implicit scheme one uses backward difference for the time derivative.
This implies

$$
\frac{\partial u(t+h,x)}{\partial t} \approx \frac{u(t+h,x) - u(t,x)}{h},
$$

thus

$$
\frac{u(t+h,x) - u(t,x)}{h} = D \frac{u(t+h,x+a) - 2u(t+h,x) + u(t+h,x-a)}{a^2}.
$$

In discretized notation

$$
u^{n+1}_k = u^n_k + r \, (u^{n+1}_{k+1} - 2u^{n+1}_k + u^{n+1}_{k-1}), \qquad k = 1 \ldots N-1.
$$

In other words, we have a system of linear equations for $u^{n+1}_i$:

$$
-r u^{n+1}_{k-1} + (1+2r) u^{n+1}_{k} - r u^{n+1}_{k+1} = u^n_k, \qquad k = 1 \ldots N-1.
$$

The system should be solved at each time step. Luckily, the system is tridiagonal, so it is solved in linear time.

In [60]:
using LinearAlgebra
function heat_implicit_iteration(u,r)
	N = length(u) - 1
	u_new = zeros(length(u))
	u_new[1] = u[1]
	u_new[N+1] = u[N+1]

    n_interior = N-1
    ld = fill(-r, n_interior-1)          # lower diagonal
    d = fill(1 + 2*r, n_interior)      # main diagonal
    ud = fill(-r, n_interior-1)    # upper diagonal
    v = copy(u[2:end-1])  # right-hand side
    v[1] += r*u[1]
    v[end] += r*u[end]

    A = Tridiagonal(ld, d, ud)
    u_new[2:N] = A \ v
    return u_new
end

function heat_implicit_solve(u₀,t,N,L,D)
    Δt = t[2]-t[1]
	Δx = L/N
	r = D*Δt/Δx^2
	u_sol = zeros(length(t),(length(u₀)))
	u = copy(u₀)
	u_sol[1,:] = u
	for i in 1:length(t)-1
		u = heat_implicit_iteration(u,r)
		u_sol[i+1,:] = u
	end
	return u_sol
end



heat_implicit_solve (generic function with 1 method)

### Implementation

In [65]:
@time begin
	# Constants
	Tleft = 50
	Tright = 0
	Tmiddle = 20
	L = 0.1      # Thickness of steel in meters
	D = 4.25e-5   # Thermal diffusivity
	N = 100      # Number of divisions in grid
	Δx = L/N       # Grid spacing
	x = 0:Δx:L   # Spatial grid
	Δt = 1e-3      # Time-step (in s)
	u = zeros(N+1)
	u[1] = Tleft
	u[N+1] = Tright
	u[2:end-1] .= Tmiddle
	r = D*Δt/Δx^2
	t_end = 10
	t = 0:Δt:t_end
    u_sol_implicit = heat_implicit_solve(u,t,N,L,D)
	u_sol_explicit = heat_FTCS_solve(u,t,N,L,D);
end;

r = 0.0425
  0.226226 seconds (290.06 k allocations: 123.450 MiB, 87.25% gc time)


In [66]:
fps = 100
Δi=round(Int, 1/ (fps*Δt))
anim = @animate for i in 1:Δi:length(t)
    p1 = heatmap(x, [1], u_sol_implicit[i, :]', xlabel="Postion", clim=(0,50))
    p2 = plot(x,u_sol_implicit[i, :],xlabel="x", ylabel="T [C°]",legend=false)
    plot!(x,u_sol_explicit[i, :],xlabel="x", ylabel="T [C°]",legend=false)
    plot(p1,p2, layout = (1,2), size=(1000,400),title="t=$(round(t[i], digits=1))")
end
mp4(anim, fps=fps)

┌ Info: Saved animation to C:\Users\dvnoe\AppData\Local\Temp\jl_kBYDUvqdy9.mp4
└ @ Plots C:\Users\dvnoe\.julia\packages\Plots\8ZnR3\src\animation.jl:156


Plots.AnimatedGif("C:\\Users\\dvnoe\\AppData\\Local\\Temp\\jl_kBYDUvqdy9.mp4")

## Crank-Nicolson scheme

Crank-Nicolson method is a combination of FTCS and implicit schemes.
Essentially it corresponds to approximating the time derivative as average of explicit and implicit methods

$$
\frac{\partial u(t,x)}{\partial t} \approx \frac{1}{2} \left[ D \, \frac{\partial^2 u(t+h,x)}{\partial x^2} + D \, \frac{\partial^2 u(t,x)}{\partial x^2} \right].
$$

Applying central differences to the spatial derivatives we obtain

$$
\frac{u(t+h,x) - u(t,x)}{h} = \frac{D}{2} \frac{u(t+h,x+a) - 2u(t+h,x) + u(t+h,x-a)}{a^2} + \frac{D}{2} \frac{u(t,x+a) - 2u(t,x) + u(t,x-a)}{a^2}.
$$

This corresponds to the following tri-diagonal system of linear equations

$$
-r u^{n+1}_{k-1} + 2(1+r) u^{n+1}_{k} - r u^{n+1}_{k+1} = ru^n_{k-1} + 2(1-r)u^n_k + ru^n_{k+1}, \qquad k = 1 \ldots N-1.
$$

### Implementation

## Heat equation in two dimensions

In two dimensions the heat equation reads

$$
\frac{\partial u}{\partial t} = D \, \left[ \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right].
$$

This equation describes the time evolution of $u(t,x,y)$ given initial profile

$$
u(t=0,x,y) = u_0(x,y),
$$

and boundary conditions
\begin{align*}
u(t,x=0,y) & = u_{\rm left}(t;y), \\
u(t,x=L,y) & = u_{\rm right}(t;y), \\
u(t,x,y=0) & = u_{\rm bottom}(t;x), \\
u(t,x,y=L) & = u_{\rm top}(t;x).
\end{align*}

Now we have to perform discretization in both $x$ and $y$ directions.
Taking the same step size $a$ in both directions, we obtain the following discretized FTCS scheme:

$$
u^{n+1}_{i,j} = u^n_{i,j} + r \, (u^n_{i+1,j} - 2u^n_{i,j} + u^n_{i-1,j})
+ r \, (u^n_{i,j+1} - 2u^n_{i,j} + u^n_{i,j-1}), \qquad i = 1 \ldots N-1, \quad j = 1 \ldots M-1.
$$

Here, as before,

$$
r \equiv \frac{Dh}{a^2},
$$

$N = L_x/a$, $M = L_y/a$, and

$$
u^n_{i,j} = u(t + hn, a*i, a*j).
$$

Implicit and Crank-Nicholson methods are also possible, but they are a bit more involved than in 1D case.
Due to the presence of more than one dimension, the linear system to be solved at each time step is larger and does not have a simple tridiagonal form.
However, sparse matrix methods can be used to solve this system efficiently.

### Heat equation in two dimensions: FTCS implementation

![](heat_equation_2D.gif)